In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

In [3]:
from collections import defaultdict
import numpy as np
import pandas as pd
import anndata
from rnasieve.preprocessing import model_from_raw_counts

In [4]:
# Download example data. See example_data.ipynb to see how it was preprocessed.
!pip3 install gdown
import gdown

!mkdir ../example_data
gdown.download('https://drive.google.com/uc?id=1U3u-mbM7dnSSINqipq-fdjrMr8Fk__rr', '../example_data/muscle_subset_bulk.h5ad')
gdown.download('https://drive.google.com/uc?id=1wKQ_XWnK2tDJNS709u707uXlj2pqnvfX', '../example_data/muscle_subset.h5ad')

You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
mkdir: ../example_data: File exists


Downloading...
From: https://drive.google.com/uc?id=1U3u-mbM7dnSSINqipq-fdjrMr8Fk__rr
To: /Users/jjhong/Documents/berkeley/Song_Lab_research/rnasieve/example_data/muscle_subset_bulk.h5ad
4.01MB [00:00, 20.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1wKQ_XWnK2tDJNS709u707uXlj2pqnvfX
To: /Users/jjhong/Documents/berkeley/Song_Lab_research/rnasieve/example_data/muscle_subset.h5ad
28.8MB [00:01, 17.0MB/s]


'../example_data/muscle_subset.h5ad'

In [5]:
# Example raw count prep
subset_bulk = anndata.read_h5ad('../example_data/muscle_subset_bulk.h5ad')
subset = anndata.read_h5ad('../example_data/muscle_subset.h5ad')

# Raw counts prep
print('Aggregating by ontology class...')
counts_by_onto_class = {}
for i in range(len(subset)):
    sc = subset[i]
    if len(sc.obs['cell_ontology_class']) == 0:
        continue
    cell_onto_class = sc.obs['cell_ontology_class'][0]
    if cell_onto_class not in counts_by_onto_class:
        counts_by_onto_class[cell_onto_class] = np.empty((sc.X.shape[1], 0), dtype=np.float32)
    counts_by_onto_class[cell_onto_class] = np.hstack(
        (counts_by_onto_class[cell_onto_class], sc.X.toarray().reshape(-1, 1)))
    
# Bulk prep
print('Aggregating bulks by age group...')
G = subset_bulk.n_vars
bulk_by_age = defaultdict(list)
for i in range(len(subset_bulk)):
    bulk = subset_bulk[i]
    if len(bulk.obs['Age']) == 0:
        continue
    age = bulk.obs['Age'][0]
    bulk_by_age[age].append(bulk.X.toarray().reshape(-1, 1))

bulk_labels = []
psis = np.empty((G, 0), dtype=np.float32)
for age in sorted(bulk_by_age.keys()):
    bulks = bulk_by_age[age]
    for i in range(len(bulks)):
        bulk_labels.append("{} months, subject {}".format(age, i))
        psis = np.hstack((psis, bulks[i]))
        
print('Done!')

Aggregating by ontology class...
Aggregating bulks by age group...
Done!


In [6]:
model, cleaned_psis = model_from_raw_counts(counts_by_onto_class, psis[:, :2])

In [7]:
model.predict(cleaned_psis)

,B cell,T cell,endothelial cell,macrophage,mesenchymal stem cell,skeletal muscle satellite cell
Bulk 0,0.027960,0.0,0.149957,0.243403,0.146383,0.432296
Bulk 1,0.037862,0.0,0.141354,0.286549,0.219244,0.314990


In [8]:
# In this example, the intervals at a significance level of 0.05 indicate the estimate is poor.
# We set sig=0.9999 for the sake of visualization.
model.compute_marginal_confidence_intervals(sig=0.9999)

[[(0.005393642170028579, 0.05052624822543157),
  (0.0, 0.02748279983220795),
  (0.1470252357621908, 0.15288973006530482),
  (0.23998256680471816, 0.24682376836458378),
  (0.14294337788513825, 0.14982338049012348),
  (0.3795769331450398, 0.4850151170874404)],
 [(0.010693553655333303, 0.06503136895604475),
  (0.0, 0.025572689198689685),
  (0.13678011026694972, 0.14592869197558264),
  (0.28112306754526917, 0.2919758723850613),
  (0.2151163198030781, 0.22337111538492777),
  (0.2527288013617421, 0.3772510986660111)]]

In [9]:
model.plot_proportions('bar').properties(title="Muscle Proportion Estimates")

alt.FacetChart(...)

In [10]:
model.plot_proportions('stacked').properties(title="Muscle Proportion Estimates")

alt.Chart(...)